In [ ]:
%%writefile nsmodule.cpp
#include <torch/extension.h>
#include <vector>

// CUDA kernels would be placed in separate .cu files; for brevity we use CPU fallback here.
at::Tensor ns_iterative_inverse(at::Tensor A, int iters) {
    auto dtype = A.dtype();
    auto device = A.device();
    TORCH_CHECK(A.dim() == 2, 
2
);
    auto I = at::eye(A.size(0), A.options());
    auto X = A.clone();
    for (int k = 0; k < iters; ++k) {
        auto X2 = 2 * X - X.matmul(A.matmul(X));
        X = X2;
    }
    return X;
}
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("ns_iterative_inverse", &ns_iterative_inverse, "Newton-Schulz iterative inverse (CPU fallback)");
}

In [ ]:
# Build the extension (requires a working PyTorch C++ build environment)
from torch.utils.cpp_extension import load
ns = load(name="nsmodule", sources=["nsmodule.cpp"], verbose=True)

In [ ]:
# Quick Python test (uses CPU fallback)
import torch
A = torch.rand(8,8)
A = A.matmul(A.t()) + 0.1 * torch.eye(8)
X = ns.ns_iterative_inverse(A, 10)
print('Residue norm:', torch.norm(A.matmul(X) - torch.eye(8)))

## Output:
```
Residue norm: 0.000123 (example)
```
: {
: {
: 
3
,
: 
,
: 

: {
: 
,
: 
3.8

: 4,
: 4